In [ ]:
import pandas as pd, numpy as np

grant_df = pd.read_csv('Grant.csv')
grant_df.columns = grant_df.iloc[0]
grant_df = grant_df[1:].drop(['Rank','Grant ID', 'Grant Number','Title','Abstract','Funding Amount','Currency',
                             'Source Linkout','Dimensions URL','RCDC Categories','HRCS HC Categories',
                              'HRCS RAC Categories','ICRP Cancer Types','ICRP CSO Categories','Start Date',
                              'End Date','Research Organization - original','Research Organization - standardized',
                             'City of Research organization','State of Research organization',
                              'Country of Research organization','Researchers',
                             'GRID ID','Resulting publications'],axis = 1)

grant_df['Funding per year'] = 10**(-5)*grant_df['Funding Amount in USD'].astype('float')/(1+grant_df['End Year'].astype('float') - grant_df['Start Year'].astype('float'))
grant_df = grant_df.drop(['Funding Amount in USD'], axis = 1)

grant_df = grant_df.dropna(subset = ['Funding per year'])

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
grant_df['FOR (ANZSRC) Categories'] = grant_df['FOR (ANZSRC) Categories'].fillna('')

for i in range(grant_df.shape[0]):
    grant_df['FOR (ANZSRC) Categories'].iloc[i] = (grant_df['FOR (ANZSRC) Categories'].iloc[i]).replace("; ",";")

tmp_df = grant_df['FOR (ANZSRC) Categories'].apply(lambda x: x.split(";"))
array_out = mlb.fit_transform(tmp_df)
df_out = pd.DataFrame(data=array_out, columns=mlb.classes_, index = grant_df.index)
df_out.columns = df_out.columns.str.replace("'"," ")

index_df = grant_df.copy()
print(index_df.shape, grant_df.shape)
grant_df = pd.concat([grant_df, df_out],axis =1).drop(['FOR (ANZSRC) Categories'],axis =1 )

grant_df =  grant_df.drop(['Funder','Units of Assessment'],axis =1)

temp_df2 = pd.get_dummies(grant_df[['Funder Group','Funder Country']])
grant_df = pd.concat([grant_df, temp_df2],axis =1).set_index(index_df.index)
print(grant_df.shape)
grant_TDA = grant_df.drop(['Title translated','Abstract translated',
                          'Start Year','Funder Group','Funder Country',
                          'Sustainable Development Goals'],axis =1)

grant_TDA['End Year'] = grant_TDA['End Year'].astype(float)
grant_TDA = grant_TDA.drop([''],axis=1)
import kmapper as km
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

mapper = km.KeplerMapper(verbose=1)


projected_X = mapper.fit_transform(grant_TDA.drop(['Funding per year'],axis =1).dropna(),
                                   projection = TSNE(), scaler = None)
from sklearn import cluster

graph = mapper.map(projected_X,
                   clusterer=cluster.DBSCAN(eps=0.1, min_samples=5, metric = 'cosine'),
                   cover=km.Cover(n_cubes=35, perc_overlap=0.3), remove_duplicate_nodes=True)



/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


(4925, 11) (4925, 11)
(4925, 165)
KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: TSNE()
	Distance matrices: False
	Scalers: None
..Projecting on data shaped (4925, 157)

..Projecting data using: 
	TSNE(verbose=1)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 4925 samples in 0.001s...


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computed neighbors for 4925 samples in 0.692s...
[t-SNE] Computed conditional probabilities for sample 1000 / 4925
[t-SNE] Computed conditional probabilities for sample 2000 / 4925
[t-SNE] Computed conditional probabilities for sample 3000 / 4925
[t-SNE] Computed conditional probabilities for sample 4000 / 4925
[t-SNE] Computed conditional probabilities for sample 4925 / 4925
[t-SNE] Mean sigma: 0.815268
[t-SNE] KL divergence after 250 iterations with early exaggeration: 74.202637
[t-SNE] KL divergence after 1000 iterations: 1.049654
Mapping on data shaped (4925, 2) using lens shaped (4925, 2)

Creating 1225 hypercubes.
No duplicate nodes found to remove.


Created 1498 edges and 607 nodes in 0:00:00.784536.


In [ ]:
funding_lst =  index_df['FOR (ANZSRC) Categories'].fillna('').apply(lambda x: x.split(";")).astype(str) + ', FPY = ' + index_df['Funding per year'].astype(str)              

html = mapper.visualize(graph,
                        path_html="grant_test.html",
                        title="Test",
                        X_names =  funding_lst.values,
                        color_function_name = ['Funding'],
                        include_searchbar = True,
                        custom_tooltips =  funding_lst.values)
from kmapper import jupyter
jupyter.display("grant_test.html")
km.draw_matplotlib(graph)
import matplotlib.pyplot as plt
plt.show()

/usr/local/lib/python3.7/dist-packages/kmapper/kmapper.py:867: UserWarning: `color_function_name` was set -- however, no `color_values` were passed, so default color_values were computed based on row order, and the passed `color_function_name` will be set as their label. This may be unexpected.
  "`color_function_name` was set -- however, no `color_values` were passed, so default color_values were computed based on row order, and the passed `color_function_name` will be set as their label. This may be unexpected."


Wrote visualization to: grant_test.html


no display found. Using non-interactive Agg backend


1       ['0102 Applied Mathematics', ' 01 Mathematical...
2       ['1117 Public Health and Health Services', ' 1...
3       ['0912 Materials Engineering', ' 09 Engineerin...
4       ['0602 Ecology', ' 0502 Environmental Science ...
5       ['0406 Physical Geography and Environmental Ge...
                              ...                        
4996                      [''], FPY = 0.10333333333333333
4997                       [''], FPY = 0.8750000000000001
4998                                   [''], FPY = 0.0775
4999                      [''], FPY = 0.12385666666666667
5000                     [''], FPY = 0.045000000000000005
Length: 4925, dtype: object